In [ ]:
# TODO: Put your names here.
# Name: Daniel Colson
# Name: Chris Kim
# Name: Aravind Karthigeyan

In [ ]:
#@title #Execute This Cell First! 
#This cell's contents should match those of photon.py
import photon
import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class Photon:

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if np.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = str(self.alpha) + "|H> "
        if np.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alpha = complex(x0, y0)
        self.beta  = complex(x1, y1)

    def prepare(self, alpha, beta, avgPhotonNumber):
        if avgPhotonNumber < 0:
            raise InputError()
        vac = photon.Photon()
        vac.prepareVacuum()
        self.alpha = alpha * np.sqrt(avgPhotonNumber) + vac.alpha
        self.beta  = beta  * np.sqrt(avgPhotonNumber) + vac.beta

    def prepareH(self, avgPhotonNumber):
        self.prepare(1, 0, avgPhotonNumber)

    def prepareV(self, avgPhotonNumber):
        self.prepare(0,1, avgPhotonNumber)

    def prepareD(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1/np.sqrt(2), avgPhotonNumber)

    def prepareA(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1/np.sqrt(2), avgPhotonNumber)

    def prepareR(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1j/np.sqrt(2), avgPhotonNumber)

    def prepareL(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1j/np.sqrt(2), avgPhotonNumber)

    def measureHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alpha)**2
        intensityV = abs(self.beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

    def measureDA(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a+b)/np.sqrt(2)
        self.beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureRL(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a-b*1j)/np.sqrt(2)
        self.beta  = (a+b*1j)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "R"
        if outcome == "V": return "L"
        else: return outcome

    def applyPolarizer(self, theta, phi):
	# Apply a polarizing filter according to the input parameters.
	# theta=0,    phi=0:     H polarizer
	# theta=pi/2, phi=0:     V polarizer
	# theta=pi/4, phi=0:     D polarizer
	# theta=pi/4, phi=pi:    A polarizer
	# theta=pi/4, phi=+pi/2: R polarizer
	# theta=pi/4, phi=-pi/2: L polarizer
        z = np.exp(1j*phi)
        a = self.alpha
        b = self.beta
        self.alpha = a*(1+np.cos(2*theta))/2 + b*np.sin(2*theta)/2*np.conj(z)
        self.beta  = a*np.sin(2*theta)/2*z + b*(1-np.cos(2*theta))/2
        # Now add an extra vacuum component.
        vac = photon.Photon()
        vac.prepareVacuum()
        a = vac.alpha
        b = vac.beta
        self.alpha = self.alpha + a*np.sin(theta)**2 + b*(-np.sin(2*theta)/2)*np.conj(z)
        self.beta  = self.beta  + a*(-np.sin(2*theta)/2)*z + b*np.cos(theta)**2

    def applyUnitaryGate(self, theta, phi, lamb):
        U = [[0,0],[0,0]]
        z1 = np.exp(1j*phi)
        z2 = -np.exp(1j*lamb)
        z3 = np.exp(1j*(lamb+phi))
        U[0][0] = np.cos(theta/2)
        U[1][0] = np.sin(theta/2)*z1
        U[0][1] = np.sin(theta/2)*z2
        U[1][1] = np.cos(theta/2)*z3
        a = self.alpha
        b = self.beta
        self.alpha = U[0][0]*a + U[0][1]*b
        self.beta  = U[1][0]*a + U[1][1]*b

    def applyXGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(np.pi, 0, np.pi)

    def applyYGate(self):
        # Applies the Pauli Y gate
        self.applyUnitaryGate(np.pi, np.pi/2, np.pi/2)

    def applyZGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(0, np.pi, 0)

    def applyHGate(self):
        # Applied the Hadamard (half-wavelength) gate
        self.applyUnitaryGate(np.pi/2, 0, np.pi)

    def applyQGate(self):
        # Applies the SH (quarter-wavelength) gate
        self.applyUnitaryGate(np.pi/2, np.pi/2, np.pi)

    def applyNoisyGate(self, p):
        # This operation acts as a depolarizing channel.
	# p = 0 leaves the photon unchanged.
	# p = 1 yields a completely random photon.
	# 0 < p < 1 yields a partially random photon.
        if p < 0 or p > 1:
            raise InputError
        theta = np.arccos(1 - 2*random.uniform(0,1)*p)
        phi   = p*(2*random.uniform(0,1) - 1)*np.pi
        lamb  = p*(2*random.uniform(0,1) - 1)*np.pi
        self.applyUnitaryGate(theta, phi, lamb)

    def applyAttenuation(self, r):
	# This operation acts as a partially reflecting beam splitter.
	# r = 0 leaves the photon unchanged.
	# r = 1 completely absorbs the photon, leaving a vacuum state.
	# 0 < r < 1 partially attenuates the photon and adds some vacuum.
	# r is the reflectivity.
        if r < 0 or r > 1:
            raise InputError
        t = np.sqrt(1-r*r) # t is the transmissivity.
        vac = photon.Photon()
        vac.prepareVacuum()
        self.alpha = (self.alpha)*t + (vac.alpha)*r
        self.beta  = (self.beta )*t + (vac.beta)*r

In [ ]:
n = 100 # number of photons

In [ ]:
# Alice --------------------------------------------

# Alice generates the raw key.
keyAlice = ""
for i in range(n): # Iterate over the number of qubits.
    # Append a random character ('0' or '1') to the end.
    if random.randint(0,1)==0: # Flip a coin (0 or 1).
        keyAlice += '0'
    else:
        keyAlice += '1'
print("keyAlice    = " + keyAlice)

keyAlice    = 0101101001101100101000010100111000111101011111000100101010110101110100001101110001101100000101111110


In [ ]:
# Alice chooses the encoding basis for each key bit.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisAlice = "" # This is just an example.
for i in range(n):
    if random.randint(0,1)==0:
        basisAlice += '+'
    else:
        basisAlice += 'x'
# TODO: Put your code here.
print("basisAlice    = " + basisAlice)

basisAlice    = ++++xxx++++xx+++++x+xx+xxxxx++x+xxxxxxxx+xxx++xxxx++++++xxx+++xx+xxx+++xxx+++xx++xxxx+xx+xx++xx++x++


In [ ]:
# Alice selects a photon state according to the key and basis.
# This should be a string of the characters 'H', 'V', 'D', 'A'.
# TODO: Put your code here.

photonAlice = ""
for i in range(n):
    if keyAlice[i] == "0":
        if basisAlice[i] == "+":
            photonAlice += "H"
        else:
            photonAlice += "D"
    else:
        if basisAlice[i] == "+":
            photonAlice += "V"
        else:
            photonAlice += "A"
print("photonAlice    = " + photonAlice)

photonAlice    = HVHVADAHHVVDAVHHVHAHDDHADADDVVAHDDAAAADAHAAAVVDDDAHHVHVHADAVHVDAVADAHHHDAAHVVADHHAADAVDDHDDVHAAVVAVH


In [ ]:
# Alice prepares and sends each photon.
# Use the methods of the Photon class to prepare each photon.
photonArray = [photon.Photon() for i in range(n)]
# TODO: Put your code here.

for i in range(n):
    if photonAlice[i] == "H":
        photonArray[i].prepareH(0.5)
    elif photonAlice[i] == "V":
        photonArray[i].prepareV(1)
    elif photonAlice[i] == "D":
        photonArray[i].prepareD(0)
    elif photonAlice[i] == "A":
        photonArray[i].prepareA(1)
    else:
        raise InputError



In [ ]:
# Eve   --------------------------------------------

# You should implement this section after completing Alice and Bob.
# Eve is allowed to do whatever she wants to the photonAlice array.
# She cannot, however, have knowledge of Alice's or Bob's choice of bases, 
# nor Bob's measurement outcomes, until they are publicly announced.

In [ ]:
# Eve selects a subsample of photons from Alice to measure.
# interceptIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=intercepted

interceptIndex = ""
# TODO: Put your code here.
for i in range(n): # Iterate over the number of qubits.
    # Append a random character ('0' or '1') to the end.
    if random.randint(0,1)==0: # Flip a coin (0 or 1).
        interceptIndex += '0'
    else:
        interceptIndex += '1'
print("interceptIndex    = " + interceptIndex)

interceptIndex    = 0111011111110101011001001000101111101010100010111111101010000000111010101101000110010111100110110010


In [ ]:
# Eve chooses a basis to measure each intercepted photon.
# basisEve should be a string of n characters.
# Use the convention '+'=H/V, 'x'=D/A, ' '=not measured
basisEve = ""
# TODO: Put your code here.
for i in range(n):
    if random.randint(0,1)==0:
        basisEve += '+'
    else:
        basisEve += 'x'
# TODO: Put your code here.
print("basisEve    = " + basisEve)

basisEve    = +x+x+++x+xxx+++x+xx+x+xxx++xxxxx+xxxx+x++xxx++xx+x+xx+xx+x+x+x+xx+x+x++x+x+++xxxx++++x+x+x+++xxxxx+x


In [ ]:
# Eve performs a measurement on each photon.
# outcomeEve should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeEve = ""
# TODO: Put your code here.
# TODO: Put your code here.
for i in range(n):
    if basisEve[i] == '+':
        outcomeEve += photonArray[i].measureHV(0.5**(1/2))
    else:
        outcomeEve += photonArray[i].measureDA(0.5**(1/2))
print("outcomeEve  = " + outcomeEve)
# This should be a string of the characters 'H', 'V', 'D', 'A'.

outcomeEve  = HMMMHVVMHAAANVMAMMMNAHNAMMNAMMDMNAMAMHAHNAAAVHDMHDMNNMMNVDVMMMHMAHMMDMNMHANVVMNDAMMHMAMANNNVHAMMANVM


In [ ]:
# Eve resends photons to Bob.
# Be sure to handle the cases in which Eve gets an invalid measurement.
# TODO: Put your code here.
# Alice prepares and sends each photon.
# Use the methods of the Photon class to prepare each photon.
photonArray2 = [photon.Photon() for i in range(n)]
# TODO: Put your code here.

for i in range(n):
    if outcomeEve[i] == "H":
        photonArray2[i].prepareH(0.5)
    elif outcomeEve[i] == "V":
        photonArray2[i].prepareV(1)
    elif outcomeEve[i] == "D":
        photonArray2[i].prepareD(0)
    elif outcomeEve[i] == "A":
        photonArray2[i].prepareA(1)

In [ ]:
# OPTIONAL: Put any other nasty tricks here.

In [ ]:
# Bob chooses a basis to measure each qubit.
# (This is similar to what Alice does.)
basisBob = ""
for i in range(n):
    if random.randint(0,1)==0:
        basisBob += '+'
    else:
        basisBob += 'x'
# TODO: Put your code here.
print("basisBob    = " + basisBob)


basisBob    = ++xx+x+x++x+x+x+xx++xxxx++x++x+x++xx+xx++++x++xx++xxxx+++xxx+x+++x++xx++xx+x++x+xxxx++xxxx+xx++xx+x+


In [ ]:
# Bob performs a measurement on each qubit.
# Use the methods of the Qubit class to measure each qubit.
outcomeBob = ""
# TODO: Put your code here.
for i in range(n):
    if basisBob[i] == '+':
        outcomeBob += photonArray[i].measureHV(0.5**(1/2))
    else:
        outcomeBob += photonArray[i].measureDA(0.5**(1/2))
print("outcomeBob  = " + outcomeBob)
# This should be a string of the characters 'H', 'V', 'D', 'A'.




outcomeBob  = MNADMMHDNNDVMHDVMMNNMDANNNDNMAHMVVNNHDDVHVMNVMNDNMADMANHVAADNDHHNDHNMNVNDMVDMMNNNMNDNVNANNNAAHMANVMM


In [ ]:
# Bob infers the raw key.
keyBob = ""
# TODO: Put your code here.
for i in range(n):
    if outcomeBob[i] == 'H':
        keyBob += '0'
    elif outcomeBob[i] == 'V':
        keyBob += '1'
    elif outcomeBob[i] == 'D':
        keyBob += '0'
    elif outcomeBob[i] == 'A':
        keyBob += '1'
    else:
        keyBob += ' '
print("keyBob      = " + keyBob)
# Only about half of Bob's raw key with match Alice's raw key.


keyBob      =   10  00  01 001     01   0  10 11  000101  1  0  10 1 01110 000 00   1 0 10       0 1 1   110 1 1  


In [ ]:
# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# Bob also announces which of his measurements were invalid.
# -----------------------------------------------------------

In [ ]:
# Alice & Bob ----------------------------------------------------------

# Alice and Bob extract their sifted keys.
# siftedAlice and siftedBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.
for i in range(n):
    if basisBob[i]==basisAlice[i]:
        siftedAlice+=keyAlice[i]
        siftedBob+=keyBob[i]
    else:
        siftedAlice+=' '
        siftedBob+=' '
        
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)

siftedAlice = 01   0  01  11 0   000 1  0 1     11 10 0  11100      10 01 0   11    0 110 1 00 110 100 0         0
siftedBob   =              0 1     0    0          00 0   1  0       0 11      0    1 0 1        0 1 1            


In [ ]:
# Alice and Bob use a portion of their sifted keys to estimate the quantum bit error rate (QBER).
# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampleIndex = ""
sampledBobQBER = 0
# TODO: Put your code here.
for i in range(n):
    if random.randint(0,1)==0:
        sampleIndex+='0'
    else:
        sampleIndex+='1'
print("sampleIndex = " + sampleIndex)
for i in range(n):
    if sampleIndex[i]=='1':
        if siftedAlice[i]!=siftedBob[i]:
            sampledBobQBER+=1
sampledBobQBER/=sampleIndex.count('1')
print("sampledBobQBER = " + str(sampledBobQBER))

sampleIndex = 0101010010001110110101000001110111100101000111101000101101100100111100100010011010110011111110111100
sampledBobQBER = 0.4339622641509434


In [ ]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""
# TODO: Put your code here.
for i in range(n):
    if sampleIndex[i]=='0':
        secureAlice+=siftedAlice[i]
        secureBob+=siftedBob[i]
print("secureAlice = " + secureAlice)
print("secureBob   = " + secureBob)

secureAlice = 0    1  0 0 1  0 1 00  0      0     11 101 1  0
secureBob   =         1      0   00  0            0      1   


In [ ]:
# Alice and Bob make a hard determination whether the channel is secure.
# If it looks like there's something fishy, better hit the kill switch!
channelSecure = True # default value, to be changed to False if Eve suspected
# TODO: Put your code here.
if sampledBobQBER>0.1:
    channelSecure=False
print("channelSecure = " + str(channelSecure))


channelSecure = False


In [ ]:
# Eve ------------------------------------------------------------------

In [ ]:
# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured
keyEve = ""
# TODO: Put your code here.
for i in range(n):
    for j in range(n):
        if basisEve[j]==basisAlice[i]:
            keyEve+=outcomeEve[j]
            break
    else:
        keyEve+=' '

In [ ]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publically revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the '0', '1', ' '=removed
stolenEve = ""
# TODO: Put your code here.
for i in range(n):
    if sampleIndex[i]=='1':
        stolenEve+=keyEve[i]
for i in range(n):
    if sampleIndex[i]=='0':
        stolenEve+=' '
print(len(stolenEve), len(secureAlice))

100 47


In [ ]:
# ANALYSIS -------------------------------------------------------------

# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
       secureKeyLengthBob += 1
       if secureAlice[i] == secureBob[i]:
           numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1-actualBobQBER) * secureKeyLengthBob / n * 1e6;

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
       stolenKeyLengthEve += 1
       if secureAlice[i] == stolenEve[i]:
           numMatchEve += 1
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1-actualEveQBER) * stolenKeyLengthEve / n * 1e6;


# DISPLAY RESULTS ------------------------------------------------------

print("")
print("basisAlice  = " + basisAlice)
print("basisBob    = " + basisBob)
print("basisEve    = " + basisEve)
print("")
print("keyAlice    = " + keyAlice)
print("keyBob      = " + keyBob)
print("keyEve      = " + keyEve)
print("")
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)
print("")
print("secureAlice = " + secureAlice)
print("secureBob   = " + secureBob)
print("stolenEve   = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob/1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve/1000) + " kbps")

# Your goal is to maximize secureKeyRateBob and minimize stolenKeyRateEve.


basisAlice  = +x+++x+x++xx+x+xx+xxxxxxxxxx++x+++++++++x++x+xx++x++++x+x+xx++++x++++xx+xx+++x+x+xxxx++xxxx++xx+x+xx
basisBob    = ++x+x+++++x++x+x+++xxxxxxxx+x+x++x+xx+xx+x+xxx+x++++xx+xx+x++++xxxxxx+x++xxxxx++x++++xxx+xxx++x+xx+x
basisEve    = xx++xxxx+xx+xxx+xx+xx++x+x++x++x+++xxx+x+x++++x++++x+x++xxxx++x+++xxxx++++++x+x++x+++xx+x+x+xxxx+++x

keyAlice    = 0010000001011010010111001011001011111000001110001110011101111110101000011010011011111011111001011100
keyBob      =   11 11000000 011 110     11  1  0 100 11010 10  0 1 10     010 0 01  1   11000011     1 11   01 10 
keyEve      = HNHHHNHNHHNNHNHNNHNNNNNNNNNNHHNHHHHHHHHHNHHNHNNHHNHHHHNHNHNNHHHHNHHHHNNHNNHHHNHNHNNNNHHNNNNHHNNHNHNN

siftedAlice = 0  0  0 010 1010 1 11100101  0101 1  0    11 0  1 10    011 111 1     01 0   11        1 11 0 011  0
siftedBob   =    1  1 000 0 01   10     1   1      0    10 1     1        010 0     1      00        1 11   01    

secureAlice =   0 0 1 100101 011 1   1  1    01 111    1 0       1 0  
secur